# FINAL PROJECT - Second-hand motorcycles recommender
___
#### MASTER IN DATA SCIENCE - KSCHOOL - 2016/17
#### KOLDO PINA ORTIZ
____

## Motivation

The aim of the present work is to get a good recommender of a second-hand vehicle, based on the adds of a second-hand marketplace web site

## Action Plan

To reach our goal, we will follow the the next steps:

1. ***Scrape*** website, **motos.net** to obtain the ***data***.
2. ***Clean*** the ***data***.
2. ***Merge*** some ***data***.
3. ***Train*** various models.
4. Compare the metrics and choose the model with the best one.
5. Create a flask web server with the app.

## Scrape

We are going to scrape each website separately.
To this end, we have developed two python scripts called ***scraper_motos.py*** and ***scrapers_coches.py***.
Both return a dataframe.

You can import them to the notebook.

In [ ]:
%pwd

In [ ]:
%cd scrapers

In [ ]:
#Call the scraper_motos function to scrape motos.net and create a csv file with the raw data in scraped_data folder
#To do that we need to change our path
# %cd scrapers
# from scraper_motos import scraper_motos
# scraper_motos()

In [ ]:
#Start time: 2017-06-13 19:28:56.441707
#num_ads 26588
#End time: 2017-06-13 22:47:15.427469

In [1]:
import pandas as pd
df_motos_raw = pd.read_csv('scraped_data/test/motos_raw_data.csv')
df_motos_raw.shape

(26081, 10)

In [2]:
df_motos_raw.head()

,city,brand,model,type,cc,color,km,year,price,url
0,Granada,TRIUMPH,SPEED TRIPLE,Naked,1050.0,Amarillo,32000.0,2006,4500,http://motos.net/ocasion/triumph/speed_triple/...
1,Vizcaya,BMW,R 1200 GS 98cv,Trail,1170.0,Blanco,74200.0,2007,8500,http://motos.net/ocasion/bmw/r_1200_gs_98cv/20...
2,Asturias,BMW,R 1200 GS,Trail,1170.0,Rojo,79000.0,2007,7500,http://motos.net/ocasion/bmw/r_1200_gs/2007-en...
3,Sevilla,HARLEY DAVIDSON,Sportster 883 XLH 53,Custom,883.0,Negro,22000.0,2003,3500,http://motos.net/ocasion/harley_davidson/sport...
4,Valencia,KYMCO,Super Dink 125i,Scooter 125cc,125.0,NaN,33000.0,2012,1950,http://motos.net/ocasion/kymco/super_dink_125i...


## Data cleaning

In [3]:
df_motos_raw['color'].fillna('not specified', inplace=True)

In [4]:
# Convert the dataframe to lower case
df_motos_raw = df_motos_raw.apply(lambda x: x.astype(str).str.lower())

In [5]:
#Join the words in the 'model' and 'type' fields with an underscorekoldo
df_motos_raw['model'] = df_motos_raw['model'].str.replace(' ', '_')
df_motos_raw['type'] = df_motos_raw['type'].str.replace(' ', '_')

In [6]:
# Looking for duplicates
df_motos_raw['is_duplicated'] = df_motos_raw.duplicated()
duplicates = df_motos_raw['is_duplicated'].sum()
print '%d duplicates' %duplicates

61 duplicates


In [7]:
# Removing duplicates and delete 'is_duplicated' column
df_motos_raw = df_motos_raw.loc[df_motos_raw['is_duplicated']==False]
df_motos_raw = df_motos_raw.drop('is_duplicated', 1)
df_motos_raw.shape

(26020, 10)

In [8]:
# Lets investigate column by column the NaNs we have in the dataframe
for column in df_motos_raw.columns:
    n_nan = df_motos_raw[column]=='nan'
    print column + " %d -- > %f" %(n_nan.sum(), (n_nan.sum()*1.0)/df_motos_raw.shape[0]*100)

city 0 -- > 0.000000
brand 0 -- > 0.000000
model 0 -- > 0.000000
type 0 -- > 0.000000
cc 97 -- > 0.372790
color 0 -- > 0.000000
km 1624 -- > 6.241353
year 0 -- > 0.000000
price 0 -- > 0.000000
url 0 -- > 0.000000


In [9]:
# Lets investigate the unique values we have in the columns
for column in ['city', 'brand', 'model', 'type', 'color', 'year']:
    column_uv = df_motos_raw[column].unique()
    print column + " --> " + "%d unique values" %len(sorted(column_uv))

city --> 52 unique values
brand --> 146 unique values
model --> 2989 unique values
type --> 15 unique values
color --> 1932 unique values
year --> 48 unique values


## Merge some data

In [10]:
#In order to calculate our first metric, we will use the following columns:
# "lon" and "lat" : These are the longitude and latitude of the corresponding city. We will add them later.
# "brand", "model", "type", "year"

In [11]:
# Calculating the longitude and latitude of the cities
import geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim()

In [12]:
cities = df_motos_raw['city'].unique()
locations_rows = []
for city in cities:
    location = geolocator.geocode([city], timeout = 15)
    locations_rows.append([city, location.latitude, location.longitude])
#Save into a csv
df_locations = pd.DataFrame(locations_rows, columns = ['city', 'lat', 'lon'])
df_locations.to_csv('auxiliary_data/locations_coords.csv', index = False)

In [13]:
df_locations.head(3)

,city,lat,lon
0,granada,37.183020,-3.602192
1,vizcaya,43.238264,-2.852208
2,asturias,43.271563,-5.853947


In [14]:
# Merge df_locations with df_motos_raw
df_motos_raw_coord = pd.merge(df_motos_raw, df_locations, on = 'city')
#Save into a csv
df_motos_raw_coord.to_csv('df_motos_raw_coord.csv', index = False)

In [16]:
#We have scraped a web to obtain a simple brand rank. 
#We have it in the fodler named scraped_data and is called rank_moto_brands.csv
#Nevertheless, not all the brands in the adds appear in it. So, first of all, we are going to add the ones that are missing.
brands_in_df_list = df_motos_raw_coord.brand.unique()
df_rank_moto_brand = pd.read_csv('scraped_data/rank_moto_brands.csv', sep=';')
rank_values = df_rank_moto_brand.values.tolist()
brands_rank_values =  df_rank_moto_brand.brand.values
for df_brand in brands_in_df_list:
    if df_brand not in brands_rank_values:
        rank_values.append([df_brand,0])
df = pd.DataFrame(rank_values, columns=['brand', 'brand_score'])
df.to_csv('auxiliary_data/rank_moto_brands.csv',  sep = ';', index=False)

In [18]:
# We have created two csv files with a rank for the motos brands and types
# called rank_motos_brands.csv and rank_motos_types.csv

# With the first one, rank_motos_brands.csv, we are gint to create another  column, with a score to the corresponding brand
df_rank_moto_brand = pd.read_csv('auxiliary_data/rank_moto_brands.csv', sep=';')
df_motos_raw_coord_brand = pd.merge(df_motos_raw_coord, df_rank_moto_brand, on = 'brand', how = 'left')
df_motos_raw_coord_brand.brand_score.fillna(0, inplace=True)
# Save into a csv
df_motos_raw_coord_brand.to_csv('df_motos_coord_brand.csv', index = False)

# With the second one, rank_moto_types.csv, we are going to create another column, with a score to the corresponding type
df_rank_moto_type = pd.read_csv('auxiliary_data/rank_moto_types.csv', sep=';')
df_motos_raw_coord_brand_types = pd.merge(df_motos_raw_coord_brand, df_rank_moto_type, on = 'type', how = 'left')
#Save into a csv
df_motos_raw_coord_brand_types.to_csv('df_motos_raw_coord_brand_type.csv', index = False)

In [19]:
df_motos_raw_coord_brand_types.head(3)

,city,brand,model,type,cc,color,km,year,price,url,lat,lon,brand_score,type_score
0,granada,triumph,speed_triple,naked,1050.0,amarillo,32000.0,2006,4500,http://motos.net/ocasion/triumph/speed_triple/...,37.18302,-3.602192,120,260
1,granada,triumph,tiger_800,trail,799.0,negro,39200.0,2011,6200,http://motos.net/ocasion/triumph/tiger_800/201...,37.18302,-3.602192,120,230
2,granada,kawasaki,vn_900_classic_special_edition,custom,903.0,negro,32000.0,2011,5000,http://motos.net/ocasion/kawasaki/vn_900_class...,37.18302,-3.602192,378,280


In [20]:
df_motos_raw_coord_brand_types.shape

(26020, 14)

In [21]:
# OK!, so we have a first version of the data we will use to recommend vehicles
# df_motos_raw_coord_brand_types
# Lets try to calculate the metric only with some fields. We are going to add these distances:
# cities distance, brand_distance, type_distance, year_distance
# We need to create some functions:

def cities_distance(city_lat, city_lon, user_lat, user_lon):
    """    
    :param city_lat: the value in the dataset's lat column to the corresponding city
    :param city_lon: the value in the dataset's lon column to the corresponding city
    :param user_lat: The corresponding lat value in the location dataset of the city selected by the user
    :param user_lon: The corresponding lon value in the location dataset of the city selected by the user
    
    :return: The value in kilometers of the distance between the two cities.
    
    Usage of the Vicenty distance
    """
    
    from geopy.distance import vincenty
    
    column_city = (city_lat, city_lon)
    user_city = (user_lat, user_lon)
    
    return (vincenty(column_city, user_city).km)

def distance_abs_value(a_value, b_value):
    return abs(a_value - b_value)

def w_s(city_row, brand_row, type_row, year_row):
    import numpy as np
    weigth = 100
    brand_weight = 40
    type_weight = 40
    year_weight = 10
    city_weight = 10
    
    params = np.array([city_row, brand_row, type_row, year_row])
    weights = np.array([city_weight, brand_weight, type_weight, year_weight])
    
    num = sum(params * weights) * 1.0
    return num/weigth

In [26]:
#Example of request of a user
user_request = ('madrid', 'bmw', 'custom', 2000)

# We need to calculate some variables:
user_lat = float(df_locations[df_locations['city'] == user_request[0]].lat)
user_lon = float(df_locations[df_locations['city'] == user_request[0]].lon)
user_brand = int(df_rank_moto_brand[df_rank_moto_brand['brand'] == user_request[1]].brand_score)
user_type = int(df_rank_moto_type[df_rank_moto_type['type'] == user_request[2]].type_score)
user_year = user_request[3]

user_vars = [user_brand, user_type, user_year]
score_columns = ['brand_score', 'type_score', 'year']

In [28]:
df_motos_raw_coord_brand_types['city_metric'] = df_motos_raw_coord_brand_types.apply(lambda row: cities_distance(row['lat'], row['lon'], user_lat, user_lon), axis=1)

for i, element in enumerate(['brand', 'type', 'year']):
    new_column = element + '_metric'
    print new_column, score_columns[i], user_vars[i]
    df_motos_raw_coord_brand_types[new_column] = df_motos_raw_coord_brand_types.apply(lambda row: distance_abs_value(int(row[score_columns[i]]), user_vars[i]), axis=1)
    
#!!! Revisar ranking de motos, falta alguna, por eso aparenden NAN en la columna brand_score al hacer el merge

df_motos_raw_coord_brand_types['total_metric_pond'] = df_motos_raw_coord_brand_types.apply(lambda row: w_s(row['city_metric'], row['brand_metric'], row['type_metric'], row['year_metric']), axis = 1)

brand_metric brand_score 356


type_metric type_score 280


year_metric year 2000


In [29]:
results = df_motos_raw_coord_brand_types.sort_values(by = ['total_metric_pond'], ascending=True)

In [30]:
results.head(5)

,city,brand,model,type,cc,color,km,year,price,url,lat,lon,brand_score,type_score,city_metric,brand_metric,type_metric,year_metric,total_metric_pond
15040,madrid,bmw,r_1200_c_classic,custom,1170.0,cobre,33000.0,2000,9800,http://motos.net/ocasion/bmw/r_1200_c_classic/...,40.416705,-3.703582,356,280,0.0,0,0,0,0.0
11571,madrid,bmw,r_1200_c_montauk,custom,1200.0,negro,31000.0,2004,8590,http://motos.net/ocasion/bmw/r_1200_c_montauk/...,40.416705,-3.703582,356,280,0.0,0,0,4,0.4
11290,madrid,bmw,r_1200_c_montauk,custom,1200.0,not specified,148000.0,2004,7400,http://motos.net/ocasion/bmw/r_1200_c_montauk/...,40.416705,-3.703582,356,280,0.0,0,0,4,0.4
12470,madrid,bmw,r_1200_cl,custom,1170.0,gris,127700.0,2004,6800,http://motos.net/ocasion/bmw/r_1200_cl/2004-en...,40.416705,-3.703582,356,280,0.0,0,0,4,0.4
15552,madrid,bmw,r_850_r,naked,848.0,not specified,45100.0,2000,2600,http://motos.net/ocasion/bmw/r_850_r/2000-en-m...,40.416705,-3.703582,356,260,0.0,0,20,0,8.0


In [ ]:
brand_null = df_motos_raw_coord_brand_types[df_motos_raw_coord_brand_types['brand_score'].isnull()]

In [ ]:
for element in brand_null.brand.unique():
    print element

In [ ]:
import numpy as np

params = np.array([2, 4, 10, 2])
weights = np.array([10, 40, 40, 10])

In [ ]:
sum(params * weights)